In [1]:
import os
os.chdir('..')

%load_ext autoreload
%autoreload 2

In [2]:
import os
from argparse import ArgumentParser, Namespace
from random import choices
import pytorch_lightning as pl
from typing import Callable, List, Optional, Sequence, Union
import squidpy as sq
import torch
from torch_geometric.loader import RandomNodeSampler
import pandas as pd
from torch_geometric.data import Data
from anndata import AnnData
from utils import adata2data
from data.node_datamodule import NodeDataModule
from models.graph_embedding import GraphEmbedding
from models.non_linear_ncem import NonLinearNCEM
from models.linear_ncem import LinearNCEM

/home/chels/anaconda3/envs/gnn/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#Mibitof
adata = sq.datasets.mibitof()
#feature_name=adata.obs.keys()[0] #Use for IMC dataset
feature_names=['Cluster','batch']
num_features=len(set(adata.obs[feature_names]))
encoder_hidden_dims=30
decoder_hidden_dims=30
num_vars=adata.X.shape[1]

#IMC
# adata = sq.datasets.imc()
# feature_name=adata.obs.keys()[0] #Use for IMC dataset
# num_features=len(set(adata.obs[feature_name]))
# num_vars=adata.X.shape[1]


In [4]:
dm = NodeDataModule(adata=adata,feature_name=feature_names,adata2data_fn=adata2data, num_workers = 8, batch_size=100)

In [5]:
dm.setup()

In [6]:
for batch in dm.train_dataloader():
    print(batch)

/home/chels/anaconda3/envs/gnn/lib/python3.9/site-packages/torch_geometric/data/collate.py:150: UserWarning: An output with one or more elements was resized since it had shape [26472], which does not match the required output shape [3309, 8]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:24.)
  value = torch.cat(values, dim=cat_dim or 0, out=out)
/home/chels/anaconda3/envs/gnn/lib/python3.9/site-packages/torch_geometric/data/collate.py:150: UserWarning: An output with one or more elements was resized since it had shape [39708], which does not match the required output shape [2, 19854]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resi

DataBatch(x=[3309, 8], edge_index=[2, 19854], y=[3309, 36], batch=[3309], ptr=[4])


In [42]:
model = LinearNCEM(in_channels=num_features,out_channels=num_vars, lr=0.0001,weight_decay=0.000001)

In [8]:
gpu=False
if gpu:
    trainer:pl.Trainer = pl.Trainer(accelerator='gpu',max_epochs=100,log_every_n_steps=1)
else:
    trainer:pl.Trainer = pl.Trainer(accelerator='cpu',max_epochs=100,log_every_n_steps=1)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [43]:
trainer.fit(model,datamodule=dm)


  | Name        | Type            | Params
------------------------------------------------
0 | model_sigma | LinearSpatial   | 324   
1 | model_mu    | LinearSpatial   | 324   
2 | loss_module | GaussianNLLLoss | 0     
------------------------------------------------
648       Trainable params
0         Non-trainable params
648       Total params
0.003     Total estimated model params size (MB)


/home/chels/anaconda3/envs/gnn/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:495: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
/home/chels/anaconda3/envs/gnn/lib/python3.9/site-packages/torch_geometric/data/collate.py:150: UserWarning: An output with one or more elements was resized since it had shape [26472], which does not match the required output shape [3309, 8]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:24.)
  value = torch.cat(values, dim=cat_dim or 0, out=out)
/home/chels/anaconda3/envs/gnn/lib/python3.9/site-packages/torch_geometric/data/collate.py:150: UserWarning: An

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])
tensor([[False,  True, False,  ..., False, False, False],
        [False,  True, False,  ..., False, False, False],
        [False,  True, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False,  True],
        [False, False, False,  ..., False,  True, False],
        [False, False, False,  ..., False,  True,  True]])
tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ..

Traceback (most recent call last):
  File "/home/chels/anaconda3/envs/gnn/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_952/2473830478.py", line 1, in <cell line: 1>
    trainer.fit(model,datamodule=dm)
  File "/home/chels/anaconda3/envs/gnn/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 770, in fit
    self._call_and_handle_interrupt(
  File "/home/chels/anaconda3/envs/gnn/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 723, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
  File "/home/chels/anaconda3/envs/gnn/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 811, in _fit_impl
    results = self._run(model, ckpt_path=self.ckpt_path)
  File "/home/chels/anaconda3/envs/gnn/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 1236, in _run
    results = self._run_